In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': "100%",
        'height': 768,
        'scroll': True,
})

{'width': '100%', 'height': 768, 'scroll': True}

# მანქანური სწავლება
## SANGU


<table align="left">
    <tr>
        <td><img src="csplus.png" align="left" style="height:100px"></td>
        <td><img src="sangu.png" align="left" style="height:100px"></td>
    </tr>
</table>


ასოცირებული პროფესორი პაატა გოგიშვილი

# ლოგისტიკური რეგრესია

# ზოგადი თეორია

მოცემულია მონაცემთა (წყვილების) სიმრავლე $X=(X^1, X^2, \cdots, X^N)$ და $Y=(Y^1, Y^2, \cdots, Y^N)$. უნდა ავაგოთ ნეირონული ქსელი რომელიც შეძლებს არსებული მონაცემების სწორ კლასიფიკაციას.

წრფივი რეგრესიისგან განსხვავებით, უნდა ავიღოთ იმდენი ნეირონი, რამდენი კლასიც გვაქვს. თითოეული ნეირონი უნდა იყოს განპირობებული შესაბამის კლასზე.

თითოეული ნეირონი უნდა იძლეოდეს მისთვის შესაბამებული კლასის არსებობის ალბათობას.

ამ მიზნის მისაღწევად, თითოეული ნეირონის მოქმედების ფუნქცია უნდა იყოს შემდეგი ხასიათის: 

$$f=\sigma{\left(\sum_{i=1}^n x_i w_i + b\right)}$$

სადაც $\sigma$ ლოგისტიკურ ფუნქციას წარმოადგენს $\sigma(z)=\frac{1}{1+e^{-z}}$



In [2]:
import numpy as np

## ზამბახების მონაცემების მომზადება

In [3]:
from sklearn.datasets import load_iris
iris = load_iris()

In [4]:
import sklearn.utils as ut 
X = iris['data']
y = iris['target']

X, y = ut.shuffle(X, y, random_state=0)


n_test = 50


X_test = X[150 - n_test : ]
y_test = y[150 - n_test : ]

X = X[:150 - n_test]
y = y[:150 - n_test]


print(f'{y}')
print(f'{X.shape}, {y.shape}')

print(f'{y_test}')
print(f'{X_test.shape}, {y_test.shape}')

print(f'{iris.feature_names}')

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1 1 1 2 0 2 0 0 1 2 2 2 2 1 2 1 1 2 2 2 2 1 2 1 0 2 1 1 1 1 2 0 0 2 1 0 0
 1 0 2 1 0 1 2 1 0 2 2 2 2 0 0 2 2 0 2 0 2 2 0 0 2 0]
(100, 4), (100,)
[0 0 1 2 2 0 0 0 1 1 0 0 1 0 2 1 2 1 0 2 0 2 0 0 2 0 2 1 1 1 2 2 1 1 0 1 2
 2 0 1 1 1 1 0 0 0 2 1 2 0]
(50, 4), (50,)
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


$b$ პარამეტრი ჯამის ნიშანში შევიტანოთ.
გამოთვლების სწორად ჩასატარებლად $X$ მასივს უნდა მივამატოთ კიდევ ერთი სვეტი.

$$z=\sum_{i=1}^n x_i w_i + b$$

$x_{n+1}=1$, $w_{n+1}=b$, 

$$z=\sum_{i=1}^{n+1} x_i w_i$$


In [5]:
X = np.c_[X, np.ones(len(X))]
X_test = np.c_[X_test, np.ones(len(X_test))]

In [6]:
import scipy.sparse.linalg as la
import scipy.optimize as opt

WW = np.array([0.1, 0.1, 0.1, 0.1, 0.1])

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = np.copy(y)

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

print(f'{y}', 1)
print(f'{label_encoder}', 2)
print(f'{onehot_encoder}', 3)
print(f'{integer_encoded}', 4)
print(f'{onehot_encoded}', 5)


values_test = np.copy(y_test)

# integer encode
label_encoder_test = LabelEncoder()
integer_encoded_test = label_encoder_test.fit_transform(values_test)
# binary encode
onehot_encoder_test = OneHotEncoder(sparse=False)
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder_test.fit_transform(integer_encoded_test)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1 1 1 2 0 2 0 0 1 2 2 2 2 1 2 1 1 2 2 2 2 1 2 1 0 2 1 1 1 1 2 0 0 2 1 0 0
 1 0 2 1 0 1 2 1 0 2 2 2 2 0 0 2 2 0 2 0 2 2 0 0 2 0] 1
LabelEncoder() 2
OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False) 3
[[2]
 [1]
 [0]
 [2]
 [0]
 [2]
 [0]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [2]
 [1]
 [0]
 [0]
 [2]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]
 [1]
 [0]
 [2]
 [2]
 [1]
 [0]
 [1]
 [1]
 [1]
 [2]
 [0]
 [2]
 [0]
 [0]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [1]
 [0]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [0]
 [0]
 [2]
 [1]
 [0]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [1]
 [2]
 [1]
 [0]
 [2]
 [2]
 [2]
 [2]
 [0]
 [0]
 [2]
 [2]
 [0]
 [2]
 [0]
 [2]
 [2]
 [0]
 [0]
 [2]
 [0]] 4
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 

In [8]:
import math

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def f(w):
    return ff(w, X, y)
    
def ff(w, x, y):
    return -np.sum(y * np.log(sigmoid(np.matmul(x, w))) + (1 - y) * np.log(sigmoid(-np.matmul(x, w))))
    
def f_prime(z):
    p = (sigmoid(np.matmul(X, z)) - y)
    return np.matmul(np.transpose(X), p)

def ca(WW):
    pass

x = np.zeros((3, 5))
    
for i in range(3):
    print()
    print(f"{i} ********************************** ")
    print()
    y = onehot_encoded[:,i]
    x[i] = opt.fmin_cg(f, WW, fprime=f_prime, callback=ca)
    print('x',x)


0 ********************************** 

Optimization terminated successfully.
         Current function value: 0.000004
         Iterations: 17
         Function evaluations: 56
         Gradient evaluations: 56
x [[  3.01480603   7.8961513  -13.96815064  -7.47464482   1.84131374]
 [  0.           0.           0.           0.           0.        ]
 [  0.           0.           0.           0.           0.        ]]

1 ********************************** 

Optimization terminated successfully.
         Current function value: 51.192458
         Iterations: 151
         Function evaluations: 332
         Gradient evaluations: 332
x [[  3.01480603   7.8961513  -13.96815064  -7.47464482   1.84131374]
 [ -0.08247043  -2.24820712   1.34312704  -2.90556124   4.81294521]
 [  0.           0.           0.           0.           0.        ]]

2 ********************************** 

Optimization terminated successfully.
         Current function value: 3.943689
         Iterations: 129
         Func

In [10]:
class Linear(object):
    """წრფივი რეგრესია"""
    
    def __init__(self, W:np.ndarray, b:float):
        self.W = W
        self.b = b
        self.dl_W = None
        self.dl_b = None
        
    def forward(self, X:np.ndarray) -> np.ndarray:
        return X @ self.W.T + self.b
    
    def loss(self, X:np.ndarray, y:np.ndarray) -> np.ndarray:
        y_pred = self.forward(X)
        lss = y_pred - y
        
        return lss
    
    def cost(self, X:np.ndarray, y:np.ndarray) -> float:
        lss = self.loss(X, y)
        cs = (1.0 / (2.0 * len(X))) * np.sum(lss**2)
        
        return cs
    
    def gradient(self, X:np.ndarray, y:np.ndarray):
        lss = self.loss(X, y)
        coef = 1.0 / len(X)
        self.dl_W = coef * np.sum(X * lss, axis=0)
        self.dl_b = coef + np.sum(lss, axis=0)
        
        return self.dl_W, self.dl_b
    
    @property
    def grad_w(self) -> np.ndarray:
        return self.dl_W
    
    @property
    def grad_b(self) -> np.ndarray:
        return self.dl_b
    
    def update(self, lr:float):
        self.W = self.W - (lr * self.dl_W)
        self.b = self.b - (lr * self.dl_b)
        
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [11]:
class Trainer(object):
    """წრფივი რეგრესიის მწვრთნელი"""
    
    def __init__(self, model: callable, lr:float=0.001, n_iters:int=10, t_iter:int=10):
        self.model = model
        self.lr = lr
        self.n_iters = n_iters
        self.t_iter = t_iter
        
    def fit(self, X:np.ndarray, y:np.ndarray):
        cs = list()
        for i in range(self.n_iters):
            c = self.model.cost(X, y)
            cs.append(c)
            d_W, d_b = self.model.gradient(X, y)
            self.model.update(self.lr)
            if i % self.t_iter == 0:
                print(f'Gradients {self.model.dl_W}, {self.model.dl_b}')
                print(f'Weights {self.model.W}, {self.model.b}')
                print(f'Iteration {i} cost = {c}')
        self.y_preds = self.model(X)
        
        return self.y_preds
    
    def validate(self, X:np.ndarray, y:np.ndarray) -> float:
        y_pred = np.rint(self.model(X))
        score = float(sum(y_pred == y)) / float(len(y))
        
        return score

In [13]:


x = np.zeros((3, 5))
W = np.zeros((1, 5))
model = Linear(W, 0.0)
trainer = Trainer(model, lr=0.0001, n_iters=300_000, t_iter=10_000)
y = np.expand_dims(y, axis=1) if len(y.shape) < 2 else y
for i in range(3):
    print()
    print(f"{i} ********************************** ")
    print()
    x[i] = trainer.fit(X, y)[1]
    print('x',x)
# print(x[0])
# print('X: ', X.shape)
# print('y: ', y.shape)


0 ********************************** 

Gradients [-2.324 -1.054 -1.951 -0.716 -0.36 ], [-35.99]
Weights [[2.324e-04 1.054e-04 1.951e-04 7.160e-05 3.600e-05]], [0.003599]
Iteration 0 cost = 0.18
Gradients [ 0.00287884 -0.01737736 -0.01557664 -0.03109214  0.00125568], [0.13556844]
Weights [[ 0.03456994 -0.00726508  0.14345558  0.08761441 -0.00458721]], [-0.46872227]
Iteration 10000 cost = 0.054884482473099856
Gradients [ 0.00674956 -0.01997065  0.00300387 -0.02142824  0.00026248], [0.03624813]
Weights [[ 0.02890135  0.01257815  0.14547065  0.11196579 -0.00514117]], [-0.53411805]
Iteration 20000 cost = 0.053784401456295906
Gradients [ 0.006932   -0.01784665  0.00358131 -0.01987463  0.00014605], [0.0246055]
Weights [[ 0.02201411  0.03151169  0.14199577  0.13252789 -0.00533525]], [-0.56352642]
Iteration 30000 cost = 0.052937147411744184
Gradients [ 6.85573957e-03 -1.58045544e-02  3.41320331e-03 -1.87464172e-02
  8.15337622e-05], [0.01815338]
Weights [[ 0.01511108  0.04831619  0.1384934   0

Gradients [ 1.09063915e-04  4.87841983e-05  1.58831560e-03 -3.81661332e-03
 -1.20104836e-04], [-0.00201048]
Weights [[-0.07619581  0.17741225  0.06440857  0.44656832 -0.00160618]], [-0.53061922]
Iteration 70000 cost = 0.04737292249668662
Gradients [ 5.04151372e-05  1.02451868e-04  1.55597962e-03 -3.67100547e-03
 -1.18897972e-04], [-0.0018898]
Weights [[-0.07627509  0.17733616  0.06283646  0.4503115  -0.00148669]], [-0.52866964]
Iteration 80000 cost = 0.047356177756427136
Gradients [-3.00166555e-06  1.50750575e-04  1.52401588e-03 -3.53264881e-03
 -1.17756525e-04], [-0.00177565]
Weights [[-0.07629837  0.17720913  0.06129649  0.45391273 -0.00136836]], [-0.52683746]
Iteration 90000 cost = 0.04734058770210748
Gradients [-5.15572786e-05  1.94129210e-04  1.49243836e-03 -3.40110569e-03
 -1.16678510e-04], [-0.00166785]
Weights [[-0.0762707   0.17703629  0.0597883   0.45737905 -0.00125115]], [-0.52511623]
Iteration 100000 cost = 0.04732604959128549
Gradients [-9.56004719e-05  2.32998016e-04  1.4

Gradients [-0.0003734   0.00041427  0.00070131 -0.00118212 -0.00010199], [-0.00019949]
Weights [[-0.06508628  0.16320766  0.02494309  0.52487735  0.00227878]], [-0.50212303]
Iteration 130000 cost = 0.04712366508776148
Gradients [-0.00036841  0.00040795  0.00068467 -0.00114949 -0.00010186], [-0.0001863]
Weights [[-0.06471536  0.16279654  0.02425013  0.52604306  0.00238071]], [-0.5019302]
Iteration 140000 cost = 0.047121489242390874
Gradients [-0.00036325  0.00040152  0.0006684  -0.00111791 -0.00010174], [-0.00017384]
Weights [[-0.06434952  0.1623918   0.02357363  0.52717667  0.00248251]], [-0.50175019]
Iteration 150000 cost = 0.04711942004415108
Gradients [-0.00035794  0.000395    0.0006525  -0.00108736 -0.00010162], [-0.00016207]
Weights [[-0.06398891  0.16199353  0.02291321  0.52827922  0.00258419]], [-0.50158229]
Iteration 160000 cost = 0.04711745209528521
Gradients [-0.00035252  0.0003884   0.00063696 -0.00105777 -0.00010151], [-0.00015095]
Weights [[-0.06363367  0.16160182  0.02226

In [14]:
www = np.array([[0.11623617, 0.11623617, 0.11623617, 0.11623617, 0.11623617],
               [0.09388506, 0.09388506, 0.09388506, 0.09388506, 0.09388506],
               [0.09387204, 0.09387204, 0.09387204, 0.09387204, 0.09387204]])
wb = np.array([-0.55124521, -0.50670914, -0.50024786])
result = X_test @ www.T + wb
result, y_test

(array([[0.59949287, 0.42275295, 0.42908534],
        [0.8900833 , 0.6574656 , 0.66376544],
        [1.08768479, 0.81707021, 0.8233479 ],
        [1.68048925, 1.29588401, 1.30209531],
        [1.80834904, 1.39915758, 1.40535455],
        [0.75059989, 0.54480353, 0.55111899],
        [0.68085819, 0.4884725 , 0.49479576],
        [0.75059989, 0.54480353, 0.55111899],
        [1.22716819, 0.92973228, 0.93599435],
        [1.35502798, 1.03300584, 1.0392536 ],
        [0.68085819, 0.4884725 , 0.49479576],
        [0.79709436, 0.58235756, 0.5886678 ],
        [1.29690989, 0.98606331, 0.99231758],
        [0.80871798, 0.59174606, 0.59805501],
        [1.83159628, 1.41793459, 1.42412896],
        [0.91333053, 0.67624262, 0.68253984],
        [1.90133798, 1.47426563, 1.48045218],
        [1.37827521, 1.05178286, 1.058028  ],
        [0.83196521, 0.61052307, 0.61682942],
        [1.37827521, 1.05178286, 1.058028  ],
        [0.86683606, 0.63868859, 0.64499103],
        [1.79672542, 1.38976907, 1

In [15]:
t = 0
for i in range(len(result)):
    if round(sum(result[i])/len(result[i])) == y_test[i]:
        t+=1
print('accuracy is:', t/len(result))

accuracy is: 0.46
